In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [112]:
from pathlib import Path
input_path = Path('/content/drive/MyDrive/Dataset')
image_files = list(input_path.glob('*.*'))
#print(image_files)
labels = [image_file.name for image_file in image_files]
labels = labels[:-1]
print(labels)

['50c.jpg', 'R1.jpg', 'R5.jpg', '5-1.jpeg', 'R2.jpg', '5-2.jpeg', '20c.jpg', '5-3.jpeg', '5-4.jpeg', '5-5.jpeg', '5-6.jpeg', '10c.jpg', '10c_O 1.png', '10c_R1.png', 'R1_O 1.png', '20c_R1.png', '20c_O 1.png', 'R1_R1.png', '50c_O1.png', 'R2_O1.png', '50c_R1.png', 'R2_R1.png', 'R5_RA.png', 'R5_O1.png', '.ipynb_checkpoints', '10c_thresholded.jpg', '10c_region_segmented.jpg', '10c_edge_segmented.jpg', 'Felzenszwalb_Segmentation_50c.jpg', 'Felzenszwalb_Segmentation_R1.jpg', 'Felzenszwalb_Segmentation_R5.jpg', 'Felzenszwalb_Segmentation_R2.jpg', 'Felzenszwalb_Segmentation_20c.jpg', 'Felzenszwalb_Segmentation_10c.jpg', 'Felzenszwalb_Segmentation_10c_O 1.png', 'Felzenszwalb_Segmentation_10c_R1.png', 'Felzenszwalb_Segmentation_R1_O 1.png', 'Felzenszwalb_Segmentation_20c_R1.png', 'Felzenszwalb_Segmentation_20c_O 1.png', 'Felzenszwalb_Segmentation_R1_R1.png', 'Felzenszwalb_Segmentation_50c_O1.png', 'Felzenszwalb_Segmentation_R2_O1.png', 'Felzenszwalb_Segmentation_50c_R1.png', 'Felzenszwalb_Segment

In [113]:
import cv2
import numpy as np
from skimage.feature import greycomatrix, greycoprops

# Load the segmented coins image
segmented_coins_path = '/content/drive/MyDrive/Dataset/10c.jpg'
segmented_coins = cv2.imread(segmented_coins_path)

# Convert the segmented coins image to grayscale
gray_coins = cv2.cvtColor(segmented_coins, cv2.COLOR_BGR2GRAY)

# Color features
color_features = segmented_coins.reshape(-1, 3)  # Reshape the image to a 2D array
mean_color = color_features.mean(axis=0)  # Calculate the mean color values

# Define a lookup table mapping color features to coin values
lookup_table = {
   # (131, 109, 73): '1 cent',
    #(201, 164, 109): '2 cents',
  #  (202, 161, 112): '5 cents',
    (184, 194, 218): '10 cents',
    (183, 197,215): '20 cents',
    (186, 201, 217): '50 cents',
    (192, 192, 194): '1 rand',
    (201, 201, 202): '2 rand',
    (193,  202, 211): '5 rand',
    (168, 185, 197):'5 rand'
}

# Find the closest color in the lookup table using Manhattan distance
closest_color = min(lookup_table, key=lambda x: np.sum(np.abs(x - mean_color)))

# Extract the value label from the lookup table
value_label = lookup_table[closest_color]

# Shape features
contours, _ = cv2.findContours(gray_coins, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

if len(contours) > 0:
    coin_contour = max(contours, key=cv2.contourArea)
    perimeter = cv2.arcLength(coin_contour, True)
else:
    perimeter = 0

# Texture features
glcm = greycomatrix(gray_coins, [1], [0], 256, symmetric=True, normed=True)
contrast = greycoprops(glcm, 'contrast')[0][0]
dissimilarity = greycoprops(glcm, 'dissimilarity')[0][0]
homogeneity = greycoprops(glcm, 'homogeneity')[0][0]
energy = greycoprops(glcm, 'energy')[0][0]
correlation = greycoprops(glcm, 'correlation')[0][0]

# Print the extracted features and value label
print('Mean color:', mean_color)
print('Perimeter:', perimeter)
print('Contrast:', contrast)
print('Dissimilarity:', dissimilarity)
print('Homogeneity:', homogeneity)
print('Energy:', energy)
print('Correlation:', correlation)
print('Value of coin:', value_label)


Mean color: [184.112925 194.5326   218.472525]
Perimeter: 796.0
Contrast: 924.9878140703518
Dissimilarity: 16.337663316582915
Homogeneity: 0.3078851168210063
Energy: 0.18387263246901808
Correlation: 0.8955283766721857
Value of coin: 10 cents


/usr/local/lib/python3.10/dist-packages/skimage/feature/__init__.py:35: skimage_deprecation: Function ``greycomatrix`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycomatrix`` instead.
  removed_version='1.0')
/usr/local/lib/python3.10/dist-packages/skimage/feature/__init__.py:42: skimage_deprecation: Function ``greycoprops`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycoprops`` instead.
  removed_version='1.0')
/usr/local/lib/python3.10/dist-packages/skimage/feature/__init__.py:42: skimage_deprecation: Function ``greycoprops`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycoprops`` instead.
  removed_version='1.0')
/usr/local/lib/python3.10/dist-packages/skimage/feature/__init__.py:42: skimage_deprecation: Function ``greycoprops`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycoprops`` instead.
  removed_version='1.0')
/usr/local/lib/python3.10/dist-packages/skimag

In [ ]:
import os
import cv2
import numpy as np
from skimage.feature import greycomatrix, greycoprops
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
# Prepare the dataset
X = []
y = []
# Specify the directory path where the coin dataset is located
dataset_dir = '/content/drive/MyDrive/Dataset/'


# Define the label mapping
label_mapping = {
    '10c.jpg': '10 cents',
    '20c.jpg': '20 cents',
    '50c.jpg': '50 cents',
    'R1.jpg': '1 rand',
    'R2.jpg': '2 rand',
    'R5.jpg': '5 rand',
    '10c_O 1.png':'10 cents',
    '10c_R1.png':'10 cents',
    '20c_O 1.png':'20 cents',
    '20c_R1.png':'20 cents',
    '5-1.jpeg': '5 rand',
    '5-2.jpeg':'5 rand',
    '5-3.jpeg':'5 rand',
    '5-4.jpeg':'5 rand',
    '5-5.jpeg':'5 rand',
    '5-6.jpeg':'5 rand',
    '50c_O1.png':'50 cents',
    '50c_R1.png':'50 cents',
    'R1_O 1.png':'1 rand',
    'R1_R1.png':'1 rand',
    'R2_O1.png':'2 rand',
    'R2_R1.png':'2 rand',
    'R5_O1.png':'5 rand',
    'R5_RA.png':'5 rand',
    'Felzenszwalb_Segmentation_10c.jpg': '10 cents',
    'Felzenszwalb_Segmentation_10c_O 1.png': '10 cents',
    'Felzenszwalb_Segmentation_10c_R1.png': '10 cents',
    'Felzenszwalb_Segmentation_10c_edge_segmented.jpg':'10 cents',
    'Felzenszwalb_Segmentation_10c_region_segmented.jpg':'10 cents',
    'Felzenszwalb_Segmentation_10c_thresholded.jpg':'10 cents',
    'Felzenszwalb_Segmentation_20c.jpg':'20 cents',
    'Felzenszwalb_Segmentation_20c_O 1.png':'20 cents',
    'Felzenszwalb_Segmentation_20c_R1.png' : '20 cents',
    'Felzenszwalb_Segmentation_50c.jpg':'50 cents',
    'Felzenszwalb_Segmentation_50c_O1.png':'50 cents',
    'Felzenszwalb_Segmentation_50c_R1.png': '50 cents',
    'Felzenszwalb_Segmentation_R1.jpg':'1 rand',
    'Felzenszwalb_Segmentation_R1_O 1.png':'1 rand',
    'Felzenszwalb_Segmentation_R1_R1.png': '1 rand',
    'Felzenszwalb_Segmentation_R2.jpg':'2 rand',
    'Felzenszwalb_Segmentation_R2_O1.png':'2 rand',
    'Felzenszwalb_Segmentation_R2_R1.png':'2 rand',
    'Felzenszwalb_Segmentation_R5.jpg':' 5 rand',
    'Felzenszwalb_Segmentation_R5_O1.png': '5 rand',
    'Felzenszwalb_Segmentation_R5_RA.png': '5 rand',
    '10c_edge_segmented.jpg': '10 cents',
    '10c_region_segmented.jpg': '10 cents',
    '10c_thresholded.jpg': '10 cents',
    'edges_10c.jpg':'10 cents',
    'edges_10c_O 1.png': '10 cents',
    'edges_10c_R1.png':'10 cents',
    'edges_10c_edge_segmented.jpg':'10 cents',
    'edges_10c_region_segmented.jpg':'10 cents',
    'edges_20c.jpg':'20 cents',
    'edges_20c_O 1.png':'20 cents',
    'edges_20c_R1.png':'20 cents',
    'edges_50c.jpg':'50 cents',
    'edges_50c_O1.png':'50 cents',
    'edges_50c_R1.png':'50 cents',
    'edges_Felzenszwalb_Segmentation_10c.jpg':'10 cents',
    'edges_Felzenszwalb_Segmentation_20c.jpg':'20 cents',
    'edges_Felzenszwalb_Segmentation_50c.jpg':'50 cents',
    'edges_Felzenszwalb_Segmentation_R1.jpg': '1 rand',
    'edges_Felzenszwalb_Segmentation_R2.jpg': '2 rand',
    'edges_Felzenszwalb_Segmentation_R2.jpg': '2 rand',
    'edges_Felzenszwalb_Segmentation_R5.jpg':'5 rand',
    'edges_R1.jpg':'1 rand',
    'edges_R2.jpg':'2 rand',
    'edges_R5.jpg':'5 rand',
    'sharpened_10c.jpg':' 10 cents',
    'sharpened_20c.jpg':' 20 cents',
    'sharpened_50c.jpg':' 50 cents',
    'sharpened_Felzenszwalb_Segmentation_10c.jpg':'10 cents',
    'sharpened_Felzenszwalb_Segmentation_20c.jpg':'20 cents',
    'sharpened_Felzenszwalb_Segmentation_50c.jpg':'50 cents',
    'sharpened_Felzenszwalb_Segmentation_R1.jpg':'1 rand',
    'sharpened_Felzenszwalb_Segmentation_R2.jpg':'2 rand',
    'sharpened_Felzenszwalb_Segmentation_R5.jpg':'5 rand',
    'sharpened_R1.jpg':'1 rand',
    'sharpened_R2.jpg':'2 rand',
    'sharpened_R5.jpg':'5 rand'

}
# Define a lookup table mapping color features to coin values
lookup_table = {
        # (131, 109, 73): '1 cent',
        # (201, 164, 109): '2 cents',
        # (202, 161, 112): '5 cents',
        (184, 194, 218): 10,
        (183, 197, 215): 20,
        (186, 201, 217): 50,
        (192, 192, 194): 1 ,
        (201, 201, 202): 2 ,
        (193, 202, 211): 5 ,
        (168, 185, 197): 5
    }
# Iterate over the images in the folder
for filename in os.listdir(dataset_dir):
    # Check if the file has an image extension
    if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
        # Load the segmented coins image
        image_path = os.path.join(dataset_dir, filename)
        segmented_coins = cv2.imread(image_path)

        # Convert the segmented coins image to grayscale
        gray_coins = cv2.cvtColor(segmented_coins, cv2.COLOR_BGR2GRAY)

        # Extract the features
        # ...

        # Color features
        color_features = segmented_coins.reshape(-1, 3)  # Reshape the image to a 2D array
        mean_color = color_features.mean(axis=0)  # Calculate the mean color values



    # Find the closest color in the lookup table using Manhattan distance
        closest_color = min(lookup_table, key=lambda x: np.sum(np.abs(x - mean_color)))

    # Extract the value label from the lookup table
        value_label = lookup_table[closest_color]

    # Shape features
        contours, _ = cv2.findContours(gray_coins, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        if len(contours) > 0:
          coin_contour = max(contours, key=cv2.contourArea)
          perimeter = cv2.arcLength(coin_contour, True)
        else:
          perimeter = 0

    # Texture features
        glcm = greycomatrix(gray_coins, [1], [0], 256, symmetric=True, normed=True)
        contrast = greycoprops(glcm, 'contrast')[0][0]
        dissimilarity = greycoprops(glcm, 'dissimilarity')[0][0]
        homogeneity = greycoprops(glcm, 'homogeneity')[0][0]
        energy = greycoprops(glcm, 'energy')[0][0]
        correlation = greycoprops(glcm, 'correlation')[0][0]
        # Get the label for the image
        label = label_mapping.get(filename)

        # Append the features and labels
        features = np.concatenate([mean_color, [perimeter, contrast, dissimilarity, homogeneity, energy, correlation,value_label]])

        X.append(features)
        y.append(label)
# Split the dataset



X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train = np.array(X_train)
# Replace None with a placeholder value
y_train = [label if label is not None else 'None' for label in y_train]

# Initialize the LabelEncoder
label_encoder = LabelEncoder()

# Fit and transform the string labels to numeric values
y_train_encoded = label_encoder.fit_transform(y_train)

#print(X_train)
# # Scale the features
scaler = StandardScaler()
X_train2 = scaler.fit_transform(X_train)
X_test2 = scaler.transform(X_test)

# # Filter out None labels from y_train
# y_train_filtered = [label for label in y_train if label is not None]

# # Get unique labels
# unique_labels = np.unique(y_train_filtered)

# print(unique_labels)
# # Train the classifier
clf = SVC()
clf = KNeighborsClassifier(n_neighbors=1)
clf.fit(X_train, y_train_encoded)

# # Predict on the test set
y_pred = clf.predict(X_test)
# Decode the predicted labels
y_pred_decoded = label_encoder.inverse_transform(y_pred)

# Replace the placeholder value with None
y_pred_decoded = [label if label != 'None' else None for label in y_pred_decoded]

# Sample test labels
#y_test = ['10 cents', '5 rand', '2 rand', None, '5 rand', '1 rand', '10 cents', '2 rand', '5 rand']

# Exclude None values from y_test and y_pred
print(y_pred)
y_test_filtered = [label for label in y_test if label is not None]
y_pred_filtered = [label for label in y_pred_decoded if label is not None]
print(y_test_filtered)


# Calculate accuracy
accuracy = accuracy_score(y_test_filtered, y_pred_filtered)
print('Accuracy:', accuracy)


In [148]:
# Calculate accuracy
from sklearn.metrics import precision_score, recall_score, f1_score

accuracy = accuracy_score(y_test_filtered, y_pred_filtered)
print('Accuracy:', accuracy)

# Calculate precision
precision = precision_score(y_test_filtered, y_pred_filtered, average='weighted')
print('Precision:', precision)

# Calculate recall
recall = recall_score(y_test_filtered, y_pred_filtered, average='weighted')
print('Recall:', recall)

# Calculate F1-score
f1 = f1_score(y_test_filtered, y_pred_filtered, average='weighted')
print('F1-score:', f1)

Accuracy: 0.625
Precision: 0.68125
Recall: 0.625
F1-score: 0.625


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [142]:
print(y_test_filtered)
print(y_pred_filtered)
print(y_pred)
print(y_test)
#print(X_train)

['2 rand', '50 cents', '5 rand', '20 cents', '50 cents', '1 rand', '5 rand', '1 rand', '2 rand', '10 cents', '50 cents', '10 cents', '10 cents', '1 rand', '50 cents', '10 cents']
['10 cents', '50 cents', '5 rand', '20 cents', '2 rand', '1 rand', '5 rand', ' 10 cents', '10 cents', '10 cents', '50 cents', '10 cents', '20 cents', '1 rand', '1 rand', '10 cents']
[5 9 8 7 6 4 8 0 5 5 9 5 7 4 4 5]
['2 rand', '50 cents', '5 rand', '20 cents', '50 cents', '1 rand', '5 rand', '1 rand', '2 rand', '10 cents', '50 cents', '10 cents', '10 cents', '1 rand', '50 cents', '10 cents']


In [146]:
from sklearn.svm import SVC

# Create an instance of the classifier
clf2 = SVC()

# Train the classifier
clf2.fit(X_train, y_train_encoded)

# # Predict on the test set
y_pred2 = clf2.predict(X_test)
# Decode the predicted labels
y_pred_decoded2 = label_encoder.inverse_transform(y_pred2)

# Replace the placeholder value with None
y_pred_decoded2 = [label if label != 'None' else None for label in y_pred_decoded2]

# Sample test labels
#y_test = ['10 cents', '5 rand', '2 rand', None, '5 rand', '1 rand', '10 cents', '2 rand', '5 rand']

# Exclude None values from y_test and y_pred
print(y_pred2)
y_test_filtered2 = [label for label in y_test if label is not None]
y_pred_filtered2 = [label for label in y_pred_decoded2 if label is not None]
print(y_test_filtered2)


# Calculate accuracy
accuracy = accuracy_score(y_test_filtered, y_pred_filtered)
print('Accuracy:', accuracy)

[5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5]
['2 rand', '50 cents', '5 rand', '20 cents', '50 cents', '1 rand', '5 rand', '1 rand', '2 rand', '10 cents', '50 cents', '10 cents', '10 cents', '1 rand', '50 cents', '10 cents']
Accuracy: 0.625


In [ ]:
import matplotlib.pyplot as plt
import cv2

# Load and process your image
image = cv2.imread('/content/drive/MyDrive/Dataset/10c_edge_segmented.jpg')
# Perform any necessary operations on the image

image2 = cv2.imread('/content/drive/MyDrive/Dataset/10c_region_segmented.jpg')
image3 = cv2.imread('/content/drive/MyDrive/Dataset/10c_region_segmented.jpg')
# Display the image using Matplotlib
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
plt.axis('off')  # Optional: Remove axis labels
plt.show()
plt.imshow(cv2.cvtColor(image2, cv2.COLOR_BGR2RGB))
plt.axis('off')  # Optional: Remove axis labels
plt.show()
plt.imshow(cv2.cvtColor(image3, cv2.COLOR_BGR2RGB))
plt.axis('off')  # Optional: Remove axis labels
plt.show()

In [ ]:
import cv2
import numpy as np
from skimage.segmentation import slic
from skimage.segmentation import felzenszwalb
from skimage.segmentation import mark_boundaries
from matplotlib import pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

# Load the image
image_path = '/content/drive/MyDrive/Dataset/10c.jpg'
image = cv2.imread(image_path)

# Convert the image to RGB
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Cluster-based segmentation using SLIC
segments_slic = slic(image_rgb, n_segments=100, compactness=10, sigma=1)

# Graph-based segmentation using Felzenszwalb's algorithm
segments_felzenszwalb = felzenszwalb(image_rgb, scale=100, sigma=0.5, min_size=50)

# Semantic segmentation using pre-trained MobileNetV2 model
# Load the pre-trained MobileNetV2 model
model = MobileNetV2(weights='imagenet', include_top=False)
input_shape = (224, 224)  # Specify the desired input shape

# Resize the image to match the input shape of the model
resized_image = cv2.resize(image_rgb, input_shape)

# Preprocess the image for the MobileNetV2 model
preprocessed_image = preprocess_input(resized_image)

# Expand dimensions to create a batch of size 1
input_image = np.expand_dims(preprocessed_image, axis=0)

# Perform semantic segmentation using the MobileNetV2 model
preds = model.predict(input_image)
segmentation_map = tf.argmax(preds, axis=-1)[0]

# Display the segmented images
fig, ax = plt.subplots(2, 2, figsize=(10, 10))
ax[0, 0].imshow(image_rgb)
ax[0, 0].set_title('Original Image')

ax[0, 1].imshow(mark_boundaries(image_rgb, segments_slic))
ax[0, 1].set_title('SLIC Segmentation')

ax[1, 0].imshow(mark_boundaries(image_rgb, segments_felzenszwalb))
ax[1, 0].set_title('Felzenszwalb Segmentation')

ax[1, 1].imshow(segmentation_map, cmap='jet')
ax[1, 1].set_title('Semantic Segmentation')

for a in ax.ravel():
    a.axis('off')

plt.tight_layout()
plt.show()


In [ ]:
import cv2
import numpy as np
from skimage.segmentation import felzenszwalb
import os

# Folder path containing the images
folder_path = '/content/drive/MyDrive/Dataset/'

# Get a list of all files in the folder
file_list = os.listdir(folder_path)

# Loop over each file in the folder
for file_name in file_list:
    # Check if the file is an image (you can add more file extensions if needed)
    if file_name.endswith('.jpg') or file_name.endswith('.png'):
        # Construct the full file path
        file_path = os.path.join(folder_path, file_name)

        # Load the image
        image = cv2.imread(file_path)

        # Convert the image to RGB
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Perform Felzenszwalb's algorithm for graph-based segmentation
        segments_felzenszwalb = felzenszwalb(image_rgb, scale=100, sigma=0.5, min_size=50)

        # Create a mask for the Felzenszwalb segments
        mask = np.zeros_like(image_rgb)

        for segment_id in np.unique(segments_felzenszwalb):
            mask[segments_felzenszwalb == segment_id] = image_rgb[segments_felzenszwalb == segment_id]

        # Save the Felzenszwalb segmentation image
        output_path = os.path.join(folder_path, f'Felzenszwalb_Segmentation_{file_name}')
        cv2.imwrite(output_path, mask)

        print(f'Felzenszwalb Segmentation image saved successfully for {file_name}.')


In [118]:
import os
import cv2

# Define the folder path
folder_path = '/content/drive/MyDrive/Dataset'

# Get the list of image files in the folder
image_files = [f for f in os.listdir(folder_path) if f.endswith(('.jpg', '.png'))]

# Loop through each image file
for file in image_files:
    # Read the image
    image_path = os.path.join(folder_path, file)
    image = cv2.imread(image_path)

    # Apply sharpening filter
    kernel_sharpening = np.array([[-1,-1,-1],
                                  [-1, 9,-1],
                                  [-1,-1,-1]])
    sharpened_image = cv2.filter2D(image, -1, kernel_sharpening)

    # Apply edge detection filter
    edges = cv2.Canny(image, 100, 200)

    # Save the filtered images
    sharpened_image_path = os.path.join(folder_path, 'sharpened_' + file)
    edges_path = os.path.join(folder_path, 'edges_' + file)
    cv2.imwrite(sharpened_image_path, sharpened_image)
    cv2.imwrite(edges_path, edges)
